In [2]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv("Dataset/Data_middle_1.csv")
df2 = pd.read_csv("Dataset/Data_middle_2.csv")
df3 = pd.read_csv("Dataset/Data_middle_3.csv")

In [5]:
concatenated_df = pd.concat([df1, df2, df3], ignore_index=True)

In [6]:
concatenated_df.nunique()

id_eleve              431875
id_annee                   5
cd_etab                  296
id_classe              27405
MoyenneGen_i1           1946
                       ...  
absence_exam_18_i1         4
absence_exam_19_i1         4
absence_exam_20_i1         4
absence_exam_23_i1         4
absence_exam_24_i1         4
Length: 72, dtype: int64

In [5]:
%pip install elasticsearch

Note: you may need to restart the kernel to use updated packages.


In [8]:
from elasticsearch import Elasticsearch

es = Elasticsearch(
    ["https://localhost:9200/"],
    basic_auth=("elastic", "+TQ3lG=S3jAfZb7RE59j"),
    verify_certs=True,
    ca_certs="C:/Users/pc/OneDrive/Documents/elasticsearch/ca.crt"  # Use the local path to the copied certificate
)

In [11]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd

# Elasticsearch authentication credentials
ELASTIC_PASSWORD = "+TQ3lG=S3jAfZb7RE59j"

# Create the Elasticsearch client with authentication
# es = Elasticsearch("https://localhost:9200/", basic_auth=("elastic", ELASTIC_PASSWORD))

es = Elasticsearch(
    "https://localhost:9200/",
    basic_auth=("elastic", ELASTIC_PASSWORD),
    verify_certs=False,  # Disable SSL certificate verification
    ssl_show_warn=False  # Suppress SSL warnings
)

# Create a generator to yield documents in the required format
def doc_generator(df1):
    for index, document in df1.iterrows():
        yield {
            "_index": "data_middle_1",
            # "_type": "_doc",
            "_id": f"doc_{index}",
            "_source": document.to_dict(),
        }

# Use helpers.bulk() to index documents in bulk
try:
    res = helpers.bulk(es, doc_generator(df1))
    print(f"Indexed {res[0]} documents")
except Exception as e:
    print("Error:", e)

# Refresh the index to make the documents available for search
es.indices.refresh(index="data_middle_1")


Indexed 349800 documents


ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

In [13]:
response = es.search(index="data_middle_1", body={"query": {"match_all": {}}})

# Process the response (e.g., convert to DataFrame)
hits = response["hits"]["hits"]
df_classes= pd.DataFrame([hit["_source"] for hit in hits])

# Display the DataFrame
df_classes.head(2)


,id_eleve,id_annee,cd_etab,id_classe,MoyenneGen_i1,NbrJourAbsenceAutorise_i1,NbrUniteAbsenceAutorise_i1,NbrJourAbsenceNonAutorise_i1,NbrUniteAbsenceNonAutorise_i1,MCaRtable_i1,...,target,id_genre,datenaiseleve,nationalite,id_handicap,Adress,Préscolarisé,profession_mere,profession_pere,Level
0,6490173,11,02450M,031891E6-6AAB-42E6-B713-CCF13807112F,10.90,56.0,0.0,0.0,0.0,0,...,0.0,1,2005,1,0.0,رقم 286بلوك 10 رباط الخير,1,inconnu,Facteur,7
1,1989718,11,02450M,031891E6-6AAB-42E6-B713-CCF13807112F,10.88,46.0,0.0,0.0,2.0,0,...,0.0,2,2006,1,0.0,دوار ناس مزيان رباط الخير,1,inconnu,utilisateur,7


Create data_middle_2

In [16]:

# Create the index if it does not exist
if not es.indices.exists(index="data_middle_2"):
    es.indices.create(index="data_middle_2")

# Create a generator to yield documents in the required format
def doc_generator(df2):
    for index, document in df2.iterrows():
        yield {
            "_index": "data_middle_2",
            "_id": f"doc_{index}",
            "_source": document.to_dict(),
        }

# Use helpers.bulk() to index documents in bulk
try:
    res = helpers.bulk(es, doc_generator(df2))
    print(f"Indexed {res[0]} documents")
except Exception as e:
    print("Error:", e)

# Refresh the index to make the documents available for search
es.indices.refresh(index="data_middle_2")


Indexed 286548 documents


ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

In [17]:
# Example query to retrieve all documents from the "classes" index
response = es.search(index="data_middle_2", body={"query": {"match_all": {}}})

# Process the response (e.g., convert to DataFrame)
hits = response["hits"]["hits"]
df_classes= pd.DataFrame([hit["_source"] for hit in hits])

# Display the DataFrame
df_classes.head(2)

,id_eleve,id_annee,cd_etab,id_classe,MoyenneGen_i1,NbrJourAbsenceAutorise_i1,NbrUniteAbsenceAutorise_i1,NbrJourAbsenceNonAutorise_i1,NbrUniteAbsenceNonAutorise_i1,MCaRtable_i1,...,target,id_genre,datenaiseleve,nationalite,id_handicap,Adress,Préscolarisé,profession_mere,profession_pere,Level
0,6048644,8,26186M,000B0B06-2DE7-4962-A248-DF7EB9487CD0,17.31,40.0,0.0,0.0,0.0,0,...,0.0,1,2002,1,0.0,حي الانارة زنقة1 درب16 رقم 173 مكناس,1,inconnu,Général,8
1,6103520,8,26186M,000B0B06-2DE7-4962-A248-DF7EB9487CD0,17.30,7.0,0.0,0.0,0.0,0,...,0.0,2,2002,1,0.0,زنقة 11 رقم 155 حي الانارة مكناس,0,inconnu,inconnu,8


Create data_middle_3

In [18]:

# Create the index if it does not exist
if not es.indices.exists(index="data_middle_3"):
    es.indices.create(index="data_middle_3")

# Create a generator to yield documents in the required format
def doc_generator(df3):
    for index, document in df3.iterrows():
        yield {
            "_index": "data_middle_3",
            "_id": f"doc_{index}",
            "_source": document.to_dict(),
        }

# Use helpers.bulk() to index documents in bulk
try:
    res = helpers.bulk(es, doc_generator(df3))
    print(f"Indexed {res[0]} documents")
except Exception as e:
    print("Error:", e)

# Refresh the index to make the documents available for search
es.indices.refresh(index="data_middle_3")

Indexed 299191 documents


ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

In [19]:
# Example query to retrieve all documents from the "classes" index
response = es.search(index="data_middle_3", body={"query": {"match_all": {}}})

# Process the response (e.g., convert to DataFrame)
hits = response["hits"]["hits"]
df_classes= pd.DataFrame([hit["_source"] for hit in hits])

# Display the DataFrame
df_classes.head(2)

,id_eleve,id_annee,cd_etab,id_classe,MoyenneGen_i1,NbrJourAbsenceAutorise_i1,NbrUniteAbsenceAutorise_i1,NbrJourAbsenceNonAutorise_i1,NbrUniteAbsenceNonAutorise_i1,MCaRtable_i1,...,target,id_genre,datenaiseleve,nationalite,id_handicap,Adress,Préscolarisé,profession_mere,profession_pere,Level
0,7677825,10,02060N,129DABC3-8BC7-40D7-9B1B-3F48B91CBF02,14.08,45.0,8.0,0.0,4.0,0,...,0.0,2,2003,1,0.0,,1,inconnu,retraité و,9
1,7330598,10,02060N,129DABC3-8BC7-40D7-9B1B-3F48B91CBF02,13.12,43.0,0.0,0.0,0.0,0,...,0.0,1,2001,1,0.0,دوار كورات,0,inconnu,inconnu,9
